***SIMULACIONES***

Simulación Baja Resolución

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from gurobipy import Model, GRB, quicksum
import networkx as nx
import warnings 
warnings.filterwarnings('ignore')
import numpy as np
import random
import gurobipy as gp

In [307]:
# LOAD DATA
#Compatibility
compatibility = pd.read_csv('/Users/valentina/Library/CloudStorage/OneDrive - UAI/TESIS/CODIGO/Datas_simulaciones/compatibilidad_total.csv', index_col=0)
compatibility.index = range(len(compatibility))
compatibility.columns = range(len(compatibility.columns))
#Pairs, recipients and donors
pairs = pd.read_csv('/Users/valentina/Library/CloudStorage/OneDrive - UAI/TESIS/CODIGO/Datas_simulaciones/parejas.csv',index_col=0)
recipients = pd.read_csv('/Users/valentina/Library/CloudStorage/OneDrive - UAI/TESIS/CODIGO/Datas_simulaciones/receptores.csv',index_col=0)
donors = pd.read_csv('/Users/valentina/Library/CloudStorage/OneDrive - UAI/TESIS/CODIGO/Datas_simulaciones/donantes.csv',index_col=0)
#Weights
hla_hr = pd.read_csv("/Users/valentina/Library/CloudStorage/OneDrive - UAI/TESIS/CODIGO/Datas_simulaciones/peso_ar.csv",index_col=0)
hla_lr = pd.read_csv("/Users/valentina/Library/CloudStorage/OneDrive - UAI/TESIS/CODIGO/Datas_simulaciones/peso_BR.csv",index_col=0)
hla_eplet = pd.read_csv("/Users/valentina/Library/CloudStorage/OneDrive - UAI/TESIS/CODIGO/Datas_simulaciones/peso_eplet.csv",index_col=0)
hla_hr.columns = hla_hr.columns.astype(int)
hla_lr.columns = hla_lr.columns.astype(int)
hla_eplet.columns = hla_eplet.columns.astype(int)

In [308]:
# Data of each locus 
base = '/Users/valentina/Library/CloudStorage/OneDrive - UAI/TESIS/CODIGO/Datas_simulaciones/locis_por_separado/BR/'
mats = {
    'A': 'mismatch_BR_A.csv',
    'B': 'mismatch_BR_B.csv',
    'C': 'mismatch_BR_C.csv',
    'DQ': 'mismatch_BR_DQ.csv',
    'DR': 'mismatch_BR_DR.csv'
}

matrix_mismatch = {}
for locus, archive in mats.items():
    matrix_mismatch[locus] = pd.read_csv(base + archive, index_col=0)

mismatch_ClassI = (
    matrix_mismatch['A'] +
    matrix_mismatch['B'] +
    matrix_mismatch['C']
)

mismatch_DQ = matrix_mismatch['DQ']
mismatch_DR = matrix_mismatch['DR']

In [309]:
# Mismatch to HLA score
HLA_DR = 2 - mismatch_DR
HLA_DQ = 2- mismatch_DQ
HLA_C1 = 6 - mismatch_ClassI

In [310]:
# Creation of initial graph with graph resolution and a minimum of quality (k)
def create_graph(pairs, compatibility, hla1, k):
    G = nx.DiGraph()
    added_edges = 0
    for i in pairs.index:
        for j in pairs.index:
            if compatibility.at[i, j] == 1 and hla1.at[i, j] >= k:
                G.add_edge(j, i, weight=hla1.at[i, j])
                added_edges += 1
    return G

In [311]:
# Changing initial weights to optimization weights on the initial graph arcs
def changing_resolution_weights(G, hla2):
    for u, v, data in G.edges(data=True):
        try:
            data['weight'] = hla2.iloc[int(v), int(u)]
        except KeyError:
            print(f"No se encontró peso para el arco ({u}, {v})")
        except Exception as e:
            print(f"Error al actualizar peso para el arco ({u}, {v}): {str(e)}")


In [312]:
# FOR OPTUNA

def optimization(G, multipliers_ethnicity, l=3, k=3):
    total_cycles = list(nx.simple_cycles(G, length_bound=3))
    valid_cycles = [cycle for cycle in total_cycles if len(cycle) <= l and all(G[u][v]['weight'] >= k for u, v in zip(cycle, cycle[1:] + cycle[:1]))]
    
    P = len(G.nodes())  
    Z = 10



    # # ---- imprimir desglose ----
    # for cycle in valid_cycles[:5]:  
    #     print(f"\nCiclo {cycle}:")
    #     term = 0
    #     for u, v in zip(cycle, cycle[1:] + cycle[:1]):
    #         eth = recipients.loc[v, 'ETHCAT']
    #         mult = multipliers_ethnicity.get(eth, 1.0)
    #         w = G[u][v]['weight']

    #         part1 = mult * 1
    #         part2 = mult * (w / (P * Z))

    #         term = term + part1 + part2


    #         print(f"  {u}->{v} | ETHCAT={eth} | multiplier={mult}")
    #         print(f"    multiplier * 1 = {part1:.6f}")
    #         print(f"    + multiplier * (w/(P*Z)) = {part2:.6f}  (w={w}, P={P}, Z={Z})")

    #     print("Term:", term )
    #     print("-" * 50)


    m = Model("optimization")
    m.setParam('OutputFlag', 0)

    x = {
        tuple(cycle): m.addVar(vtype=GRB.BINARY, 
                               name=f"x_{'_'.join(map(str, cycle))}") 
        for cycle in valid_cycles
    }

    m.setObjective(
        quicksum(
            x[tuple(cycle)] * 
            #(
                sum(
                multipliers_ethnicity.get(recipients.loc[v, 'ETHCAT'], 1.0) *
                (1 + (G[u][v]['weight'] / (P * Z)))
                for u, v in zip(cycle, cycle[1:] + cycle[:1])
            )
            #)/P
            
            for cycle in valid_cycles
        ),
        GRB.MAXIMIZE
    )

    for i in G.nodes():
        m.addConstr(quicksum(x[tuple(cycle)] for cycle in valid_cycles if i in cycle) <= 1, 
                    name=f"node_usage_{i}")

    try:
        m.optimize()
    except gp.GurobiError as e:
        print(f"⚠️ Error of Gurobi: {e}")
      
        return None, [], True  

    G_optimal = nx.DiGraph()
    selected_cycles = []

    if m.status == GRB.OPTIMAL:
        for cycle in valid_cycles:
            if x[tuple(cycle)].X > 0.5:
                selected_cycles.append(cycle)
                for i in range(len(cycle)):
                    u, v = cycle[i], cycle[(i + 1) % len(cycle)]
                    G_optimal.add_edge(u, v, weight=G[u][v]['weight'])


    return G_optimal, selected_cycles, False


In [ ]:
# Complete definition of the simulation

def update_matrices(indexes, df):
    return df.iloc[indexes, indexes]
frequiency_waiting_nodes = {}

def run_simulation(total_time, arrival_rate, departure_rate, match_run, pairs, compatibility, hla_lr, hla_hr, hla_eplet, multipliers_ethnicity, seed= None):
    # Evaluation at different resolutions
    if seed is not None:
        np.random.seed(seed)
        
    lr_quality_ethcat1 = []
    lr_quality_ethcat2 = []
    lr_quality_ethcat4 = []
    lr_quality_ethcat5 = []
    lr_quality_ethcat6 = []
    lr_quality_ethcat7 = []
    
    hr_quality_ethcat1 = []
    hr_quality_ethcat2 = []
    hr_quality_ethcat4 = []
    hr_quality_ethcat5 = []
    hr_quality_ethcat6 = []
    hr_quality_ethcat7 = []

    e_quality_ethcat1 = []
    e_quality_ethcat2 = []
    e_quality_ethcat4 = []
    e_quality_ethcat5 = []
    e_quality_ethcat6 = []
    e_quality_ethcat7 = []

    waiting_list = []
    waiting_times = {}  
    arrivals_by_ethcat = {}
    historial_cycles = []
    historial_departures = []
    cont=0
    available_indexes = set(pairs.index)

    
    departures_by_ethcat = {1: [], 2: [], 4: [], 5: [], 6: [], 7: []}
    HLA_classI_ethcat = {1: [], 2: [], 4: [], 5: [], 6: [], 7: []}
    HLA_DR_ethcat = {1: [], 2: [], 4: [], 5: [], 6: [], 7: []}
    HLA_DQ_ethcat = {1: [], 2: [], 4: [], 5: [], 6: [], 7: []}

    # Simulation per month
    for month in range(total_time):
     
        selected_cycles = []
       
        new_entries = np.random.poisson(arrival_rate)
        new_entries = min(new_entries, len(available_indexes))
        new_index = np.random.choice(list(available_indexes), size=new_entries, replace=False)
        cont+= len(new_index)
        waiting_list.extend(new_index)
        available_indexes.difference_update(new_index)

        for idx in new_index:
            waiting_times[idx] = {'arrival': month}
            ethnicity = recipients.loc[recipients['Nodo'] == idx, 'ETHCAT'].iloc[0]
            if ethnicity in arrivals_by_ethcat:
                arrivals_by_ethcat[ethnicity] += 1
            else:
                arrivals_by_ethcat[ethnicity] = 1
        
 
        departure = np.random.poisson(departure_rate)
        departure_indexes = [] 
        if departure:
            departure = min(departure, len(waiting_list))
            departure_indexes = np.random.choice(waiting_list, size=departure, replace=False)
            waiting_list = [idx for idx in waiting_list if idx not in departure_indexes]
            historial_departures.extend(departure_indexes)

        for idx in departure_indexes:
            ethnicity = recipients.loc[idx, 'ETHCAT']
            departures_by_ethcat[ethnicity].append(idx)

        if (month + 1) % match_run == 0:
            waiting_list_index = waiting_list.copy()
            df_waiting_list = pairs.loc[waiting_list_index]
            
            filtered_compatibility = update_matrices(waiting_list_index, compatibility)
            filtered_weight = update_matrices(waiting_list_index, hla_lr) # Insert the resolution data that you want for create the graph
            
    
            G = create_graph(df_waiting_list, filtered_compatibility, filtered_weight, k= 3) # Insert the minimum weight for create the graph 
            changing_resolution_weights(G, hla_lr) # Insert the resolution data that you want for the optimization

            
            
            G_optimal, selected_cycles,  error_gurobi = optimization(G, multipliers_ethnicity, l=3, k=3 ) # Insert the minimum quality that you want for the cycles 

            if error_gurobi:
                return float("inf"), None

            for u, v, data in G_optimal.edges(data=True):
                node_ethcat = recipients.loc[v, 'ETHCAT'] 
                
                # HLA for differents loci (optimization resolution)
                #value_HLA_classI = HLA_C1.iloc[v, u]
                value_HLA_DR = HLA_DR.iloc[v, u]
                value_HLA_DQ = HLA_DQ.iloc[v, u]

                #HLA_classI_ethcat[node_ethcat].append(value_HLA_classI)
                HLA_DR_ethcat[node_ethcat].append(value_HLA_DR)
                HLA_DQ_ethcat[node_ethcat].append(value_HLA_DQ)


                if node_ethcat == 1:
                    lr_quality_ethcat1.append(hla_lr.iloc[v, u])
                    hr_quality_ethcat1.append(hla_hr.iloc[v, u])
                    e_quality_ethcat1.append(hla_eplet.iloc[v, u])
                elif node_ethcat == 2:
                    lr_quality_ethcat2.append(hla_lr.iloc[v, u])
                    hr_quality_ethcat2.append(hla_hr.iloc[v, u])
                    e_quality_ethcat2.append(hla_eplet.iloc[v, u])
                elif node_ethcat == 4:
                    lr_quality_ethcat4.append(hla_lr.iloc[v, u])
                    hr_quality_ethcat4.append(hla_hr.iloc[v, u])
                    e_quality_ethcat4.append(hla_eplet.iloc[v, u])
                elif node_ethcat == 5:
                    lr_quality_ethcat5.append(hla_lr.iloc[v, u])
                    hr_quality_ethcat5.append(hla_hr.iloc[v, u])
                    e_quality_ethcat5.append(hla_eplet.iloc[v, u])
                elif node_ethcat == 6:
                    lr_quality_ethcat6.append(hla_lr.iloc[v, u])
                    hr_quality_ethcat6.append(hla_hr.iloc[v, u])
                    e_quality_ethcat6.append(hla_eplet.iloc[v, u])
                elif node_ethcat == 7:
                    lr_quality_ethcat7.append(hla_lr.iloc[v, u])
                    hr_quality_ethcat7.append(hla_hr.iloc[v, u])
                    e_quality_ethcat7.append(hla_eplet.iloc[v, u])
            historial_cycles.extend(selected_cycles)

            nodes_in_cycles = [node for cycle in selected_cycles for node in cycle]
            waiting_list = [idx for idx in waiting_list if idx not in nodes_in_cycles]
            for idx in nodes_in_cycles:
                waiting_times[idx]['departure'] = month

    nodes_in_historial = [node for cycle in historial_cycles for node in cycle]

    data = []

    for idx in nodes_in_historial:
        if idx in waiting_times:
            waiting_time = waiting_times[idx]['departure'] - waiting_times[idx]['arrival']
            ethnicity = recipients.loc[idx, 'ETHCAT'] 
            data.append({'node': idx, 'waiting time (months)': waiting_time, 'ethnicity': ethnicity})

    

    df_ethnicity = pd.DataFrame(data)
    df_ethnicity['ethnicity'] = df_ethnicity['ethnicity'].astype(int)
    df_ethnicity1 = df_ethnicity[df_ethnicity['ethnicity'] == 1]
    df_ethnicity2 = df_ethnicity[df_ethnicity['ethnicity'] == 2]
    df_ethnicity4 = df_ethnicity[df_ethnicity['ethnicity'] == 4]
    df_ethnicity5 = df_ethnicity[df_ethnicity['ethnicity'] == 5]
    df_ethnicity6 = df_ethnicity[df_ethnicity['ethnicity'] == 6]
    df_ethnicity7 = df_ethnicity[df_ethnicity['ethnicity'] == 7]
    waiting_time_mean_1 = df_ethnicity1['waiting time (months)'].mean()
    waiting_time_mean_2 = df_ethnicity2['waiting time (months)'].mean()
    waiting_time_mean_4 = df_ethnicity4['waiting time (months)'].mean()
    waiting_time_mean_5 = df_ethnicity5['waiting time (months)'].mean()
    waiting_time_mean_6 = df_ethnicity6['waiting time (months)'].mean()
    waiting_time_mean_7 = df_ethnicity7['waiting time (months)'].mean()

    
    outgoing_per_ethcat = {e: len(departures_by_ethcat[e]) for e in [1,2,4,5,6,7]}
    L_per_ethcat  = {e: outgoing_per_ethcat [e] / arrivals_by_ethcat.get(e, 1) for e in outgoing_per_ethcat } 
    F_per_ethcat  = {e: len([c for c in historial_cycles for n in c if recipients.loc[n,'ETHCAT']==e]) / arrivals_by_ethcat.get(e, 1)
                   for e in [1,2,4,5,6,7]}  
    
     
    total_trasplants = sum(len(cycle) for cycle in historial_cycles)
    total_entries = sum(arrivals_by_ethcat.values())
    F_total = total_trasplants / total_entries if total_entries > 0 else 0


     

    return {
        'historial_cycles': historial_cycles,
        'historial_departures': historial_departures,
        'entries_nodes': cont,
        'arrivals_by_ethcat': arrivals_by_ethcat,
        'waiting_time_mean1':   waiting_time_mean_1 ,
        'waiting_time_mean2':   waiting_time_mean_2,
        'waiting_time_mean4':   waiting_time_mean_4,
        'waiting_time_mean5':   waiting_time_mean_5,
        'waiting_time_mean6':   waiting_time_mean_6,
        'waiting_time_mean7':   waiting_time_mean_7,
        'lr_quality_ethcat1': np.mean(lr_quality_ethcat1),
        'lr_quality_ethcat2': np.mean(lr_quality_ethcat2),
        'lr_quality_ethcat4': np.mean(lr_quality_ethcat4),
        'lr_quality_ethcat5': np.mean(lr_quality_ethcat5),
        'lr_quality_ethcat6': np.mean(lr_quality_ethcat6),
        'lr_quality_ethcat7': np.mean(lr_quality_ethcat7),

        'hr_quality_ethcat1': np.mean(hr_quality_ethcat1),
        'hr_quality_ethcat2': np.mean(hr_quality_ethcat2),
        'hr_quality_ethcat4': np.mean(hr_quality_ethcat4),
        'hr_quality_ethcat5': np.mean(hr_quality_ethcat5),
        'hr_quality_ethcat6': np.mean(hr_quality_ethcat6),
        'hr_quality_ethcat7': np.mean(hr_quality_ethcat7),

        'e_quality_ethcat1': np.mean(e_quality_ethcat1),
        'e_quality_ethcat2': np.mean(e_quality_ethcat2),
        'e_quality_ethcat4': np.mean(e_quality_ethcat4),
        'e_quality_ethcat5': np.mean(e_quality_ethcat5),
        'e_quality_ethcat6': np.mean(e_quality_ethcat6),
        'e_quality_ethcat7': np.mean(e_quality_ethcat7),
        'avg_ClassI_HLA_ethcat1': np.mean(HLA_classI_ethcat[1]), 
        'avg_ClassI_HLA_ethcat2': np.mean(HLA_classI_ethcat[2]),
        'avg_ClassI_HLA_ethcat4': np.mean(HLA_classI_ethcat[4]),
        'avg_ClassI_HLA_ethcat5': np.mean(HLA_classI_ethcat[5]),
        'avg_ClassI_HLA_ethcat6': np.mean(HLA_classI_ethcat[6]),
        'avg_ClassI_HLA_ethcat7': np.mean(HLA_classI_ethcat[7]), 
        'HLA_ClassI_total': np.mean(
                                    HLA_classI_ethcat[1] + 
                                    HLA_classI_ethcat[2] + 
                                    HLA_classI_ethcat[4] + 
                                    HLA_classI_ethcat[5] + 
                                    HLA_classI_ethcat[6] + 
                                    HLA_classI_ethcat[7]
                                ),

        'avg_DR_HLA_ethcat1': np.mean(HLA_DR_ethcat[1]), 
        'avg_DR_HLA_ethcat2': np.mean(HLA_DR_ethcat[2]),
        'avg_DR_HLA_ethcat4': np.mean(HLA_DR_ethcat[4]),
        'avg_DR_HLA_ethcat5': np.mean(HLA_DR_ethcat[5]),
        'avg_DR_HLA_ethcat6': np.mean(HLA_DR_ethcat[6]),
        'avg_DR_HLA_ethcat7': np.mean(HLA_DR_ethcat[7]), 
        'HLA_DR_total': np.mean(
                                    HLA_DR_ethcat[1] + 
                                    HLA_DR_ethcat[2] + 
                                    HLA_DR_ethcat[4] + 
                                    HLA_DR_ethcat[5] + 
                                    HLA_DR_ethcat[6] + 
                                    HLA_DR_ethcat[7]
                                ),

        'avg_DQ_HLA_ethcat1': np.mean(HLA_DQ_ethcat[1]), 
        'avg_DQ_HLA_ethcat2': np.mean(HLA_DQ_ethcat[2]),
        'avg_DQ_HLA_ethcat4': np.mean(HLA_DQ_ethcat[4]),
        'avg_DQ_HLA_ethcat5': np.mean(HLA_DQ_ethcat[5]),
        'avg_DQ_HLA_ethcat6': np.mean(HLA_DQ_ethcat[6]),
        'avg_DQ_HLA_ethcat7': np.mean(HLA_DQ_ethcat[7]), 
        'HLA_DQ_total': np.mean(
                                    HLA_DQ_ethcat[1] + 
                                    HLA_DQ_ethcat[2] + 
                                    HLA_DQ_ethcat[4] + 
                                    HLA_DQ_ethcat[5] + 
                                    HLA_DQ_ethcat[6] + 
                                    HLA_DQ_ethcat[7]
                                ),

        'F_per_ethcat': F_per_ethcat,        
        'L_per_ethcat' : L_per_ethcat,          
        'F_total': F_total,
        'lr_quality_total': np.mean(
                                    lr_quality_ethcat1 + 
                                    lr_quality_ethcat2 + 
                                    lr_quality_ethcat4 + 
                                    lr_quality_ethcat5 + 
                                    lr_quality_ethcat6 + 
                                    lr_quality_ethcat7
                                ),
        'hr_quality_total': np.mean(
                                    hr_quality_ethcat1 + 
                                    hr_quality_ethcat2 + 
                                    hr_quality_ethcat4 + 
                                    hr_quality_ethcat5 + 
                                    hr_quality_ethcat6 + 
                                    hr_quality_ethcat7
                                ),
        'e_quality_total': np.mean(
                                    e_quality_ethcat1 + 
                                    e_quality_ethcat2 + 
                                    e_quality_ethcat4 + 
                                    e_quality_ethcat5 + 
                                    e_quality_ethcat6 + 
                                    e_quality_ethcat7
                                ),
        'Waiting_time_mean': pd.concat([
                                    df_ethnicity1['waiting time (months)'],
                                    df_ethnicity2['waiting time (months)'],
                                    df_ethnicity4['waiting time (months)'],
                                    df_ethnicity5['waiting time (months)'],
                                    df_ethnicity6['waiting time (months)'],
                                    df_ethnicity7['waiting time (months)']
                                    ]).mean()


        }

In [ ]:
####### For the same seed just in the iterations ######
def run_simulation(
    total_time,
    arrival_rate,
    departure_rate,
    match_run,
    pairs,
    compatibility,
    hla_lr,
    hla_hr,
    hla_eplet,
    multipliers_ethnicity,
    seed=None
):
    if seed is None:
        rng_arr = np.random.RandomState()   # arrivals
        rng_dep = np.random.RandomState()   # departures
    else:
        rng_arr = np.random.RandomState(seed + 1) 
        rng_dep = np.random.RandomState(seed + 2)  

    # accumulators
    lr_quality_ethcat1, lr_quality_ethcat2, lr_quality_ethcat4 = [], [], []
    lr_quality_ethcat5, lr_quality_ethcat6, lr_quality_ethcat7 = [], [], []
    hr_quality_ethcat1, hr_quality_ethcat2, hr_quality_ethcat4 = [], [], []
    hr_quality_ethcat5, hr_quality_ethcat6, hr_quality_ethcat7 = [], [], []
    e_quality_ethcat1,  e_quality_ethcat2,  e_quality_ethcat4  = [], [], []
    e_quality_ethcat5,  e_quality_ethcat6,  e_quality_ethcat7  = [], [], []

    waiting_list = []
    waiting_times = {}
    arrivals_by_ethcat = {}
    historial_cycles = []
    historial_departures = []
    cont = 0

    departures_by_ethcat = {1: [], 2: [], 4: [], 5: [], 6: [], 7: []}
    HLA_classI_ethcat = {1: [], 2: [], 4: [], 5: [], 6: [], 7: []}
    HLA_DR_ethcat     = {1: [], 2: [], 4: [], 5: [], 6: [], 7: []}
    HLA_DQ_ethcat     = {1: [], 2: [], 4: [], 5: [], 6: [], 7: []}

    pool = np.array(pairs.index, copy=True)
    pool = rng_arr.permutation(pool)
    ptr = 0  

    # simulation month per month
    for month in range(total_time):
        selected_cycles = []

        # arrivals
        new_entries = int(rng_arr.poisson(arrival_rate))
        remaining = len(pool) - ptr
        if remaining <= 0:
            new_entries = 0
        else:
            new_entries = min(new_entries, remaining)

        if new_entries > 0:
            new_index = pool[ptr:ptr + new_entries]
            ptr += new_entries
            cont += new_entries
            waiting_list.extend(new_index.tolist())

            for idx in new_index:
                waiting_times[idx] = {'arrival': month}
                ethnicity = recipients.loc[recipients['Nodo'] == idx, 'ETHCAT'].iloc[0]
                arrivals_by_ethcat[ethnicity] = arrivals_by_ethcat.get(ethnicity, 0) + 1

        # departures
        departure = int(rng_dep.poisson(departure_rate))
        if departure > 0 and len(waiting_list) > 0:
            departure = min(departure, len(waiting_list))
            pos = rng_dep.choice(len(waiting_list), size=departure, replace=False)
            pos.sort()
            departure_indexes = [waiting_list[p] for p in pos]
            keep = set(departure_indexes)
            waiting_list = [idx for idx in waiting_list if idx not in keep]
            historial_departures.extend(departure_indexes)

            for idx in departure_indexes:
                ethnicity = recipients.loc[idx, 'ETHCAT']
                departures_by_ethcat[ethnicity].append(idx)

        # match run
        if (month + 1) % match_run == 0 and len(waiting_list) > 0:
            waiting_list_index = waiting_list.copy()
            df_waiting_list = pairs.loc[waiting_list_index]

            filtered_compatibility = update_matrices(waiting_list_index, compatibility)
            filtered_weight = update_matrices(waiting_list_index, hla_eplet)

            G = create_graph(df_waiting_list, filtered_compatibility, filtered_weight, k=45)
            changing_resolution_weights(G, hla_eplet)

            G_optimal, selected_cycles, error_gurobi = optimization(G, multipliers_ethnicity, l=3, k=45)
            if error_gurobi:
                return float("inf"), None

            for u, v, data in G_optimal.edges(data=True):
                node_ethcat = recipients.loc[v, 'ETHCAT']

                value_HLA_classI = HLA_C1.iloc[v, u]
                value_HLA_DR     = HLA_DR.iloc[v, u]
                value_HLA_DQ     = HLA_DQ.iloc[v, u]

                HLA_classI_ethcat[node_ethcat].append(value_HLA_classI)
                HLA_DR_ethcat[node_ethcat].append(value_HLA_DR)
                HLA_DQ_ethcat[node_ethcat].append(value_HLA_DQ)

                if node_ethcat == 1:
                    lr_quality_ethcat1.append(hla_lr.iloc[v, u]); hr_quality_ethcat1.append(hla_hr.iloc[v, u]); e_quality_ethcat1.append(hla_eplet.iloc[v, u])
                elif node_ethcat == 2:
                    lr_quality_ethcat2.append(hla_lr.iloc[v, u]); hr_quality_ethcat2.append(hla_hr.iloc[v, u]); e_quality_ethcat2.append(hla_eplet.iloc[v, u])
                elif node_ethcat == 4:
                    lr_quality_ethcat4.append(hla_lr.iloc[v, u]); hr_quality_ethcat4.append(hla_hr.iloc[v, u]); e_quality_ethcat4.append(hla_eplet.iloc[v, u])
                elif node_ethcat == 5:
                    lr_quality_ethcat5.append(hla_lr.iloc[v, u]); hr_quality_ethcat5.append(hla_hr.iloc[v, u]); e_quality_ethcat5.append(hla_eplet.iloc[v, u])
                elif node_ethcat == 6:
                    lr_quality_ethcat6.append(hla_lr.iloc[v, u]); hr_quality_ethcat6.append(hla_hr.iloc[v, u]); e_quality_ethcat6.append(hla_eplet.iloc[v, u])
                elif node_ethcat == 7:
                    lr_quality_ethcat7.append(hla_lr.iloc[v, u]); hr_quality_ethcat7.append(hla_hr.iloc[v, u]); e_quality_ethcat7.append(hla_eplet.iloc[v, u])

            historial_cycles.extend(selected_cycles)

            nodes_in_cycles = [node for cycle in selected_cycles for node in cycle]
            waiting_list = [idx for idx in waiting_list if idx not in nodes_in_cycles]
            for idx in nodes_in_cycles:
                waiting_times[idx]['departure'] = month

    # some metrics
    nodes_in_historial = [node for cycle in historial_cycles for node in cycle]
    data = []
    for idx in nodes_in_historial:
        if idx in waiting_times:
            waiting_time = waiting_times[idx]['departure'] - waiting_times[idx]['arrival']
            ethnicity = recipients.loc[idx, 'ETHCAT']
            data.append({'node': idx, 'waiting time (months)': waiting_time, 'ethnicity': ethnicity})

    df_ethnicity = pd.DataFrame(data)
    if not df_ethnicity.empty:
        df_ethnicity['ethnicity'] = df_ethnicity['ethnicity'].astype(int)
        df_ethnicity1 = df_ethnicity[df_ethnicity['ethnicity'] == 1]
        df_ethnicity2 = df_ethnicity[df_ethnicity['ethnicity'] == 2]
        df_ethnicity4 = df_ethnicity[df_ethnicity['ethnicity'] == 4]
        df_ethnicity5 = df_ethnicity[df_ethnicity['ethnicity'] == 5]
        df_ethnicity6 = df_ethnicity[df_ethnicity['ethnicity'] == 6]
        df_ethnicity7 = df_ethnicity[df_ethnicity['ethnicity'] == 7]
        waiting_time_mean_1 = df_ethnicity1['waiting time (months)'].mean()
        waiting_time_mean_2 = df_ethnicity2['waiting time (months)'].mean()
        waiting_time_mean_4 = df_ethnicity4['waiting time (months)'].mean()
        waiting_time_mean_5 = df_ethnicity5['waiting time (months)'].mean()
        waiting_time_mean_6 = df_ethnicity6['waiting time (months)'].mean()
        waiting_time_mean_7 = df_ethnicity7['waiting time (months)'].mean()
    else:
        waiting_time_mean_1 = waiting_time_mean_2 = waiting_time_mean_4 = np.nan
        waiting_time_mean_5 = waiting_time_mean_6 = waiting_time_mean_7 = np.nan

    outgoing_per_ethcat = {e: len(departures_by_ethcat[e]) for e in [1,2,4,5,6,7]}
    L_per_ethcat  = {e: outgoing_per_ethcat[e] / arrivals_by_ethcat.get(e, 1) for e in outgoing_per_ethcat}
    F_per_ethcat  = {e: len([c for c in historial_cycles for n in c if recipients.loc[n, 'ETHCAT'] == e]) / arrivals_by_ethcat.get(e, 1)
                     for e in [1,2,4,5,6,7]}

    total_trasplants = sum(len(cycle) for cycle in historial_cycles)
    total_entries = sum(arrivals_by_ethcat.values())
    F_total = total_trasplants / total_entries if total_entries > 0 else 0

    return {
        'historial_cycles': historial_cycles,
        'historial_departures': historial_departures,
        'entries_nodes': cont,
        'arrivals_by_ethcat': arrivals_by_ethcat,
        'waiting_time_mean1': waiting_time_mean_1,
        'waiting_time_mean2': waiting_time_mean_2,
        'waiting_time_mean4': waiting_time_mean_4,
        'waiting_time_mean5': waiting_time_mean_5,
        'waiting_time_mean6': waiting_time_mean_6,
        'waiting_time_mean7': waiting_time_mean_7,

        'lr_quality_ethcat1': np.mean(lr_quality_ethcat1),
        'lr_quality_ethcat2': np.mean(lr_quality_ethcat2),
        'lr_quality_ethcat4': np.mean(lr_quality_ethcat4),
        'lr_quality_ethcat5': np.mean(lr_quality_ethcat5),
        'lr_quality_ethcat6': np.mean(lr_quality_ethcat6),
        'lr_quality_ethcat7': np.mean(lr_quality_ethcat7),

        'hr_quality_ethcat1': np.mean(hr_quality_ethcat1),
        'hr_quality_ethcat2': np.mean(hr_quality_ethcat2),
        'hr_quality_ethcat4': np.mean(hr_quality_ethcat4),
        'hr_quality_ethcat5': np.mean(hr_quality_ethcat5),
        'hr_quality_ethcat6': np.mean(hr_quality_ethcat6),
        'hr_quality_ethcat7': np.mean(hr_quality_ethcat7),

        'e_quality_ethcat1': np.mean(e_quality_ethcat1),
        'e_quality_ethcat2': np.mean(e_quality_ethcat2),
        'e_quality_ethcat4': np.mean(e_quality_ethcat4),
        'e_quality_ethcat5': np.mean(e_quality_ethcat5),
        'e_quality_ethcat6': np.mean(e_quality_ethcat6),
        'e_quality_ethcat7': np.mean(e_quality_ethcat7),

        'avg_ClassI_HLA_ethcat1': np.mean(HLA_classI_ethcat[1]),
        'avg_ClassI_HLA_ethcat2': np.mean(HLA_classI_ethcat[2]),
        'avg_ClassI_HLA_ethcat4': np.mean(HLA_classI_ethcat[4]),
        'avg_ClassI_HLA_ethcat5': np.mean(HLA_classI_ethcat[5]),
        'avg_ClassI_HLA_ethcat6': np.mean(HLA_classI_ethcat[6]),
        'avg_ClassI_HLA_ethcat7': np.mean(HLA_classI_ethcat[7]),
        'HLA_ClassI_total': np.mean(HLA_classI_ethcat[1] + HLA_classI_ethcat[2] + HLA_classI_ethcat[4] +
                                    HLA_classI_ethcat[5] + HLA_classI_ethcat[6] + HLA_classI_ethcat[7]),

        'avg_DR_HLA_ethcat1': np.mean(HLA_DR_ethcat[1]),
        'avg_DR_HLA_ethcat2': np.mean(HLA_DR_ethcat[2]),
        'avg_DR_HLA_ethcat4': np.mean(HLA_DR_ethcat[4]),
        'avg_DR_HLA_ethcat5': np.mean(HLA_DR_ethcat[5]),
        'avg_DR_HLA_ethcat6': np.mean(HLA_DR_ethcat[6]),
        'avg_DR_HLA_ethcat7': np.mean(HLA_DR_ethcat[7]),
        'HLA_DR_total': np.mean(HLA_DR_ethcat[1] + HLA_DR_ethcat[2] + HLA_DR_ethcat[4] +
                                 HLA_DR_ethcat[5] + HLA_DR_ethcat[6] + HLA_DR_ethcat[7]),

        'avg_DQ_HLA_ethcat1': np.mean(HLA_DQ_ethcat[1]),
        'avg_DQ_HLA_ethcat2': np.mean(HLA_DQ_ethcat[2]),
        'avg_DQ_HLA_ethcat4': np.mean(HLA_DQ_ethcat[4]),
        'avg_DQ_HLA_ethcat5': np.mean(HLA_DQ_ethcat[5]),
        'avg_DQ_HLA_ethcat6': np.mean(HLA_DQ_ethcat[6]),
        'avg_DQ_HLA_ethcat7': np.mean(HLA_DQ_ethcat[7]),
        'HLA_DQ_total': np.mean(HLA_DQ_ethcat[1] + HLA_DQ_ethcat[2] + HLA_DQ_ethcat[4] +
                                 HLA_DQ_ethcat[5] + HLA_DQ_ethcat[6] + HLA_DQ_ethcat[7]),

        'F_per_ethcat': F_per_ethcat,
        'L_per_ethcat': L_per_ethcat,
        'F_total': F_total,
        'lr_quality_total': np.mean(lr_quality_ethcat1 + lr_quality_ethcat2 + lr_quality_ethcat4 +
                                    lr_quality_ethcat5 + lr_quality_ethcat6 + lr_quality_ethcat7),
        'hr_quality_total': np.mean(hr_quality_ethcat1 + hr_quality_ethcat2 + hr_quality_ethcat4 +
                                    hr_quality_ethcat5 + hr_quality_ethcat6 + hr_quality_ethcat7),
        'e_quality_total': np.mean(e_quality_ethcat1 + e_quality_ethcat2 + e_quality_ethcat4 +
                                   e_quality_ethcat5 + e_quality_ethcat6 + e_quality_ethcat7),
        'Waiting_time_mean': pd.concat([
            df_ethnicity[df_ethnicity['ethnicity'] == e]['waiting time (months)'] for e in [1,2,4,5,6,7]
        ]).mean() if not df_ethnicity.empty else np.nan
    }


In [ ]:
def objective_function(m1, m2, m4, m5, base_seed=12345):
    
    multipliers_ethnicity = {
    1: m1,
    2: m2,
    4: m4,
    5: m5,
    6: 1.0,
    7: 1.0
}

    simulations_results = []
    for i in range(100):
        #print(f"Runing simulation {i + 1} of 100")
        seed_i = base_seed + i

        results = run_simulation(
            total_time=10*12, 
            arrival_rate=(990/(10*12)), 
            departure_rate=((990/(10*12))*0.29), 
            match_run=3,
            pairs=pairs, 
            compatibility=compatibility, 
            hla_lr = hla_lr,
            hla_hr=hla_hr,
            hla_eplet= hla_eplet,
            multipliers_ethnicity= multipliers_ethnicity,
            seed=seed_i  
        )
    
        simulations_results.append({
            'simulation': i + 1,
            'cycles': results['historial_cycles'],
            'total_cycles': len(results['historial_cycles']),
            'total_departures': len(results['historial_departures']),
            'total_entries': results['entries_nodes'],
            'arrivals_by_ethcat': results['arrivals_by_ethcat'],
            'waiting_time_mean1':   results['waiting_time_mean1'] ,
            'waiting_time_mean2':   results['waiting_time_mean2'],
            'waiting_time_mean4':   results['waiting_time_mean4'],
            'waiting_time_mean5':   results['waiting_time_mean5'],
            'waiting_time_mean6':   results['waiting_time_mean6'],
            'waiting_time_mean7':   results['waiting_time_mean7'],
            'lr_quality_ethcat1': results['lr_quality_ethcat1'],
            'lr_quality_ethcat2': results['lr_quality_ethcat2'],
            'lr_quality_ethcat4': results['lr_quality_ethcat4'],
            'lr_quality_ethcat5': results['lr_quality_ethcat5'],
            'lr_quality_ethcat6': results['lr_quality_ethcat6'],
            'lr_quality_ethcat7': results['lr_quality_ethcat7'],
            'hr_quality_ethcat1': results['hr_quality_ethcat1'],
            'hr_quality_ethcat2': results['hr_quality_ethcat2'],
            'hr_quality_ethcat4': results['hr_quality_ethcat4'],
            'hr_quality_ethcat5': results['hr_quality_ethcat5'],
            'hr_quality_ethcat6': results['hr_quality_ethcat6'],
            'hr_quality_ethcat7': results['hr_quality_ethcat7'],
            'e_quality_ethcat1': results['e_quality_ethcat1'],
            'e_quality_ethcat2': results['e_quality_ethcat2'],
            'e_quality_ethcat4': results['e_quality_ethcat4'],
            'e_quality_ethcat5': results['e_quality_ethcat5'],
            'e_quality_ethcat6': results['e_quality_ethcat6'],
            'e_quality_ethcat7': results['e_quality_ethcat7'],
            'avg_ClassI_HLA_ethcat1': results['avg_ClassI_HLA_ethcat1'],
            'avg_ClassI_HLA_ethcat2': results['avg_ClassI_HLA_ethcat2'],
            'avg_ClassI_HLA_ethcat4': results['avg_ClassI_HLA_ethcat4'],
            'avg_ClassI_HLA_ethcat5': results['avg_ClassI_HLA_ethcat5'],
            'avg_ClassI_HLA_ethcat6': results['avg_ClassI_HLA_ethcat6'],
            'avg_ClassI_HLA_ethcat7': results['avg_ClassI_HLA_ethcat7'],
            'HLA_ClassI_total': results['HLA_ClassI_total'],

            'avg_DR_HLA_ethcat1': results['avg_DR_HLA_ethcat1'],
            'avg_DR_HLA_ethcat2': results['avg_DR_HLA_ethcat2'],
            'avg_DR_HLA_ethcat4': results['avg_DR_HLA_ethcat4'],
            'avg_DR_HLA_ethcat5': results['avg_DR_HLA_ethcat5'],
            'avg_DR_HLA_ethcat6': results['avg_DR_HLA_ethcat6'],
            'avg_DR_HLA_ethcat7': results['avg_DR_HLA_ethcat7'],
            'HLA_DR_total': results['HLA_DR_total'],
            

            'avg_DQ_HLA_ethcat1': results['avg_DQ_HLA_ethcat1'],
            'avg_DQ_HLA_ethcat2': results['avg_DQ_HLA_ethcat2'],
            'avg_DQ_HLA_ethcat4': results['avg_DQ_HLA_ethcat4'],
            'avg_DQ_HLA_ethcat5': results['avg_DQ_HLA_ethcat5'],
            'avg_DQ_HLA_ethcat6': results['avg_DQ_HLA_ethcat6'],
            'avg_DQ_HLA_ethcat7': results['avg_DQ_HLA_ethcat7'],
            'HLA_DQ_total': results['HLA_DQ_total'],

            'F_per_ethcat':   results['F_per_ethcat'],   
            'L_per_ethcat':   results['L_per_ethcat'],   
            'F_total': results['F_total'],
            'lr_quality_total': results['lr_quality_total'],
            'hr_quality_total': results['hr_quality_total'],
            'e_quality_total': results['e_quality_total'],
            'Waiting_time_mean': results['Waiting_time_mean']
        })
        
        
    df_results = pd.DataFrame(simulations_results)
    z = 10 #HERE change the z. 10 for LR and HR, 138 for eplet.
    F_P = np.mean(df_results['F_total'])
    HLA_P = np.mean(df_results['lr_quality_total']) /z #HERE change the resolution
    
    results_ethnicity = []
    

    for index, row in df_results.iterrows():
        nodes = set(nodo for ciclo in row['cycles'] for nodo in ciclo)
        df_nodes = recipients[recipients['Nodo'].isin(nodes)]
        count_ethnicity = df_nodes['ETHCAT'].value_counts().to_dict()
        results_ethnicity .append({'simulation': row['simulation'], **count_ethnicity})

    df_results_ethnicity  = pd.DataFrame(results_ethnicity ).fillna(0)
    arrivals_by_ethcat = pd.DataFrame(df_results['arrivals_by_ethcat'].tolist())
    arrivals_by_ethcat_mean = arrivals_by_ethcat.mean()
    target_ethnicity = [1, 2, 4, 5, 6, 7]
    m = sum(arrivals_by_ethcat_mean[e] for e in target_ethnicity) 
    target_ethnicity = [1, 2, 4, 5]
    transplant_by_ethcat_mean = df_results_ethnicity.mean()
    percentage_by_ethnicity = (transplant_by_ethcat_mean) / arrivals_by_ethcat_mean
    quality = df_results[
        ['lr_quality_ethcat1', 'lr_quality_ethcat2', 'lr_quality_ethcat4', 'lr_quality_ethcat5'] #HERE change the resolution
    ].mean()
    quality = quality / z

    kpi_total = 0
    rows = []

    for e in target_ethnicity:
        
        s = arrivals_by_ethcat_mean[e]
        F_s = percentage_by_ethnicity[e]
        HLA_s = quality[f'lr_quality_ethcat{e}'] #HERE change the resolution
        ######
        #F_diff = (s / m)*(F_s - F_P)
        #HLA_diff = (s / m)*((1/m)*(HLA_s - HLA_P))

        F_diff = F_s - F_P
        HLA_diff = (HLA_s - HLA_P)/m
        #####

        term = (s / m) * (abs(F_s - F_P) + (1/m)*abs(HLA_s - HLA_P))
        #term = (s / m) * (abs(F_s - F_P) + abs(HLA_s - HLA_P))

        kpi_total = kpi_total + term
        print(f"For ethcat {e} : s = {s}, m = {m}, F_s = {F_s}, F_P = {F_P}, HLA_s = {HLA_s}, HLA_P = {HLA_P} ")

        rows.append({
        "ethcat": e,
        "kpi_total": term,      
        "F_diff": F_diff,         
        "HLA_diff": HLA_diff    
    })

    tabla_kpi = pd.DataFrame(rows).set_index("ethcat")
    print("KPI:",kpi_total)
    print("\nResumen KPI por etnia (fila = etnia):")
    print(tabla_kpi.to_string(float_format=lambda x: f"{x:.6f}"))

    return tabla_kpi



In [210]:
tabla = objective_function(1,1,1,1.000)

For ethcat 1 : s = 591.56, m = 978.0099999999999, F_s = 0.6987625938197309, F_P = 0.6709550906277216, HLA_s = 0.470233449273064, HLA_P = 0.45119669872974166 
For ethcat 2 : s = 170.05, m = 978.0099999999999, F_s = 0.6408115260217583, F_P = 0.6709550906277216, HLA_s = 0.41555242105152573, HLA_P = 0.45119669872974166 
For ethcat 4 : s = 147.22, m = 978.0099999999999, F_s = 0.623013177557397, F_P = 0.6709550906277216, HLA_s = 0.43156936271385027, HLA_P = 0.45119669872974166 
For ethcat 5 : s = 55.31, m = 978.0099999999999, F_s = 0.6085698788645814, F_P = 0.6709550906277216, HLA_s = 0.3937720627235929, HLA_P = 0.45119669872974166 
KPI: 0.03283010246464941

Resumen KPI por etnia (fila = etnia):
        kpi_total    F_diff  HLA_diff
ethcat                               
1        0.016831  0.027808  0.000019
2        0.005248 -0.030144 -0.000036
4        0.007220 -0.047942 -0.000020
5        0.003531 -0.062385 -0.000059


In [208]:
tabla

,kpi_total,F_diff,HLA_diff
ethcat,,,
1,0.016945,0.027995,0.000020
2,0.006334,-0.036389,-0.000037
4,0.008173,-0.054276,-0.000021
5,0.001737,-0.030649,-0.000057


**OPTION 1 SEARCH JUST WITH "F" DIFFERENCE**

In [ ]:
import numpy as np
import pandas as pd

def _kpi_total_from_table(tabla: pd.DataFrame) -> float:
    #Sum of per-ethnicity KPI. 
    if 'kpi_total' not in tabla.columns:
        raise ValueError("The table must have a 'kpi_total' column.")
    return float(tabla['kpi_total'].sum())

def _get_F_col(tabla: pd.DataFrame) -> str:
    #Return F-diff column 
    if 'F_diff' in tabla.columns:
        return 'F_diff'
    if 'F_s - F_P' in tabla.columns:
        return 'F_s - F_P'
    raise ValueError("The table must have an F column: 'F_diff' or 'F_s - F_P'.")

def _state_key(m: dict, round_decimals: int = 6):
    #key for the multiplier state (rounded to avoid float noise)
    return tuple(round(m[e], round_decimals) for e in (1,2,4,5))

def _eval_state(m: dict, base_seed: int, cache: dict, state_round: int):
    #Evaluate objective_function
    key = _state_key(m, state_round)
    if key in cache:
        return cache[key]
    tabla = objective_function(m[1], m[2], m[4], m[5], base_seed=base_seed)
    kpi = _kpi_total_from_table(tabla)
    cache[key] = (tabla, kpi)
    return tabla, kpi

# algorithm
def single_push_search(step=0.001, max_iters=200, f_zero_tol=1e-12, state_round=6, base_seed=12345, verbose=True):
    """
     - Pick ethnicity with largest |F_diff|.
      - If F_diff > 0 → decrease its multiplier by 'step'; if < 0 → increase it.
      - Stop if the next state was already visited.
      - Return the best state (lowest KPI) seen.
    """
    # Initial multipliers
    m = {1:1.0, 2:1.0, 4:1.0, 5:1.0}

    cache = {}
    tabla, kpi = _eval_state(m, base_seed, cache, state_round)
    F_col = _get_F_col(tabla)

    if verbose:
        print(f"[init] KPI = {kpi:.6f}  m = {m}  | base_seed={base_seed}")

    history = [(0, m.copy(), kpi)]
    seen = { _state_key(m, state_round) }

    best_m, best_kpi, best_table = m.copy(), kpi, tabla.copy()

    for it in range(1, max_iters+1):
        # Select ethnicity with the largest absolute F deviation
        F_series = tabla[F_col].replace([np.inf, -np.inf], np.nan).dropna()
        if F_series.empty or F_series.abs().max() <= f_zero_tol:
            if verbose:
                print(f"[iter {it}] No useful signal in F (all ~0). Stop.")
            break

        eth_star = int(F_series.abs().idxmax())
        F_val = float(F_series.loc[eth_star])

        # Direction: positive → take, negative → give
        step_dir = -1.0 if F_val > 0.0 else (1.0 if F_val < 0.0 else 0.0)
        if step_dir == 0.0:
            if verbose:
                print(f"[iter {it}] Top |F| is exactly 0. Stop.")
            break

        proposed = m.copy()
        proposed[eth_star] += step_dir * float(step)

        # Stop if this state was already explored
        key = _state_key(proposed, state_round)
        if key in seen:
            if verbose:
                print(f"[iter {it}] Next step would revisit a previous state. Stop.")
            break

        # Evaluate proposed state
        new_tabla, new_kpi = _eval_state(proposed, base_seed, cache, state_round)

        if verbose:
            action = "give" if step_dir > 0 else "take"
            print(f"[iter {it}] {action} step={step:.3f} to ethcat {eth_star} "
                  f"| KPI {kpi:.6f} → {new_kpi:.6f} | m={proposed}")

        m, tabla, kpi = proposed, new_tabla, new_kpi
        F_col = _get_F_col(tabla)
        seen.add(key)
        history.append((it, m.copy(), kpi))

        # actualizar mejor
        if new_kpi < best_kpi:
            best_kpi = new_kpi
            best_m = m.copy()
            best_table = tabla.copy()

    return {
        "best_multipliers": best_m,
        "best_kpi": best_kpi,
        "best_table": best_table,
        "last_multipliers": m,        
        "last_kpi": kpi,
        "last_table": tabla,
        "history": history,           
        "visited_count": len(seen),
    }

res = single_push_search(step=0.001, max_iters=200, base_seed=12345, f_zero_tol=1e-12, state_round=6, verbose=True)
print("Best m:", res["best_multipliers"])
print("Best KPI:", res["best_kpi"])
print(res["best_table"])


For ethcat 1 : s = 591.56, m = 978.0099999999999, F_s = 0.6639225099736291, F_P = 0.639818133553713, HLA_s = 0.7670647309753491, HLA_P = 0.761172272632183 
For ethcat 2 : s = 170.05, m = 978.0099999999999, F_s = 0.6397530138194648, F_P = 0.639818133553713, HLA_s = 0.7532620420466656, HLA_P = 0.761172272632183 
For ethcat 4 : s = 147.22, m = 978.0099999999999, F_s = 0.5846352397772041, F_P = 0.639818133553713, HLA_s = 0.7512880067846092, HLA_P = 0.761172272632183 
For ethcat 5 : s = 55.31, m = 978.0099999999999, F_s = 0.538239016452721, F_P = 0.639818133553713, HLA_s = 0.7414409854376514, HLA_P = 0.761172272632183 
KPI: 0.028650185956190802

Resumen KPI por etnia (fila = etnia):
        kpi_total    F_diff  HLA_diff
ethcat                               
1        0.014583  0.024104  0.000006
2        0.000013 -0.000065 -0.000008
4        0.008308 -0.055183 -0.000010
5        0.005746 -0.101579 -0.000020
[init] KPI = 0.028650  m = {1: 1.0, 2: 1.0, 4: 1.0, 5: 1.0}  | base_seed=12345
For et

**OPTION 2 SEARCH WITH "F" DIFFERENCE AND "HLA" DIFFERENCE**

In [ ]:
import numpy as np
import pandas as pd

def _kpi_total_from_table(tabla: pd.DataFrame) -> float:
    #Sum of per-ethnicity KPI. 
    if 'kpi_total' not in tabla.columns:
        raise ValueError("The table must have a 'kpi_total' column.")
    return float(tabla['kpi_total'].sum())

def _get_F_col(tabla: pd.DataFrame) -> str:
    #Resolve the F-difference column
    if 'F_diff' in tabla.columns: return 'F_diff'
    if 'F_s - F_P' in tabla.columns: return 'F_s - F_P'
    raise ValueError("The table must have an F column: 'F_diff' or 'F_s - F_P'.")

def _get_H_col(tabla: pd.DataFrame) -> str:
    # Resolve the HLA-difference column
    if 'HLA_diff' in tabla.columns: return 'HLA_diff'
    if 'HLA_s - HLA_P' in tabla.columns: return 'HLA_s - HLA_P'
    raise ValueError("The table must have an HLA column: 'HLA_diff' or 'HLA_s - HLA_P'.")

def _state_key(m: dict, round_decimals: int = 6):
    return tuple(round(m[e], round_decimals) for e in (1,2,4,5))

def _eval_state(m: dict, base_seed: int, cache: dict, state_round: int):
    key = _state_key(m, state_round)
    if key in cache:
        return cache[key]
    # objective_function 
    tabla = objective_function(m[1], m[2], m[4], m[5], base_seed=base_seed)
    kpi = _kpi_total_from_table(tabla)
    cache[key] = (tabla, kpi)
    return tabla, kpi

# algorithm
def single_push_search_both(step=0.001, max_iters=200, zero_tol=1e-12,
                            state_round=6, base_seed=12345, verbose=True):
    """
      1) Read F and HLA diffs from the table.
      2) Pick (ethnicity, signal) with largest |diff|.
      3) If diff > 0 → TAKE 'step' from that ethnicity's multiplier.
         If diff < 0 → GIVE 'step' to that ethnicity's multiplier.
      4) Accept unless the next state was already visited (then stop).
    Returns the best (lowest KPI) among all visited states.
    """
    # Initial multipliers
    m = {1:1.0, 2:1.0, 4:1.0, 5:1.0}
    cache = {}

    tabla, kpi = _eval_state(m, base_seed, cache, state_round)
    F_col, H_col = _get_F_col(tabla), _get_H_col(tabla)

    if verbose:
        print(f"[init] KPI = {kpi:.6f}  m = {m}  | base_seed={base_seed}")

    history = [(0, m.copy(), kpi)]
    seen = { _state_key(m, state_round) }
    best_m, best_kpi, best_table = m.copy(), kpi, tabla.copy()

    for it in range(1, max_iters+1):
        F_series  = tabla[F_col].replace([np.inf, -np.inf], np.nan)
        H_series  = tabla[H_col].replace([np.inf, -np.inf], np.nan)
        long = pd.concat(
            [F_series.rename('F'), H_series.rename('HLA')], axis=1
        ).stack().dropna()          # Multiindex: (ethcat, signal)

        if long.empty or long.abs().max() <= zero_tol:
            if verbose:
                print(f"[iter {it}] No useful signal in F/HLA (all ~0). Stop.")
            break

        # # Pick the max absolute deviation (ethnicity, which signal)
        idx_star = long.abs().idxmax()   # (ethcat, 'F' or 'HLA')
        eth_star, sig_star = int(idx_star[0]), idx_star[1]
        diff_val = float(long.loc[idx_star])

        # Step direction
        step_dir = -1.0 if diff_val > 0.0 else (1.0 if diff_val < 0.0 else 0.0)
        if step_dir == 0.0:
            if verbose:
                print(f"[iter {it}] Top |diff| is exactly 0. Stop.")
            break

        # Propose next multipliers
        proposed = m.copy()
        proposed[eth_star] += step_dir * float(step)

        # Stop if we would revisit a previous state
        key = _state_key(proposed, state_round)
        if key in seen:
            if verbose:
                print(f"[iter {it}] Next step would revisit a previous state. Stop.")
            break

        # Evaluate proposed state
        new_tabla, new_kpi = _eval_state(proposed, base_seed, cache, state_round)

        if verbose:
            action = "give" if step_dir > 0 else "take"
            print(f"[iter {it}] {action} step={step:.3f} to ethcat {eth_star} (signal={sig_star}, diff={diff_val:+.6f}) "
                  f"| KPI {kpi:.6f} → {new_kpi:.6f} | m={proposed}")

        # Accept, record, and update best if improved
        m, tabla, kpi = proposed, new_tabla, new_kpi
        F_col, H_col = _get_F_col(tabla), _get_H_col(tabla)
        seen.add(key)
        history.append((it, m.copy(), kpi))

        if new_kpi < best_kpi:
            best_kpi = new_kpi
            best_m = m.copy()
            best_table = tabla.copy()

    return {
        "best_multipliers": best_m,
        "best_kpi": best_kpi,
        "best_table": best_table,
        "last_multipliers": m,
        "last_kpi": kpi,
        "last_table": tabla,
        "history": history,
        "visited_count": len(seen),
    }

res = single_push_search_both(step=0.001, max_iters=200,
                              zero_tol=1e-12, state_round=6,
                              base_seed=12345, verbose=True)
print("Best m:", res["best_multipliers"])
print("Best KPI:", res["best_kpi"])
print(res["best_table"])


For ethcat 1 : s = 591.56, m = 978.0099999999999, F_s = 0.6639225099736291, F_P = 0.639818133553713, HLA_s = 0.7670647309753491, HLA_P = 0.761172272632183 
For ethcat 2 : s = 170.05, m = 978.0099999999999, F_s = 0.6397530138194648, F_P = 0.639818133553713, HLA_s = 0.7532620420466656, HLA_P = 0.761172272632183 
For ethcat 4 : s = 147.22, m = 978.0099999999999, F_s = 0.5846352397772041, F_P = 0.639818133553713, HLA_s = 0.7512880067846092, HLA_P = 0.761172272632183 
For ethcat 5 : s = 55.31, m = 978.0099999999999, F_s = 0.538239016452721, F_P = 0.639818133553713, HLA_s = 0.7414409854376514, HLA_P = 0.761172272632183 
KPI: 0.028650185956190802

Resumen KPI por etnia (fila = etnia):
        kpi_total    F_diff  HLA_diff
ethcat                               
1        0.014583  0.024104  0.000006
2        0.000013 -0.000065 -0.000008
4        0.008308 -0.055183 -0.000010
5        0.005746 -0.101579 -0.000020
[init] KPI = 0.028650  m = {1: 1.0, 2: 1.0, 4: 1.0, 5: 1.0}  | base_seed=12345
For et

**After of knowing m1, m2, m3 and m4**

In [ ]:
#Insert the multipliers found 

multipliers_ethnicity = {
    1: 1,
    2: 1.001,
    4: 1.003,
    5: 1.004,
    6: 1.0,
    7: 1.0
}

simulations_results = []
#base_seed=12345

for i in range(100):
    #seed_i = base_seed + i
    print(f"Runing simulation {i + 1} of 100")
    results = run_simulation(
        total_time=10*12, 
        arrival_rate=(990/(10*12)), 
        departure_rate=((990/(10*12))*0.29), 
        match_run=3,
        pairs=pairs, 
        compatibility=compatibility, 
        hla_lr = hla_lr,
        hla_hr=hla_hr,
        hla_eplet= hla_eplet,
        multipliers_ethnicity= multipliers_ethnicity,
        #seed=seed_i  
    )
   
    simulations_results.append({
        'simulacion': i + 1,
        'cycles': results['historial_cycles'],
        'total_cycles': len(results['historial_cycles']),
        'total_departures': len(results['historial_departures']),
        'total_entries': results['entries_nodes'],
        'arrivals_by_ethcat': results['arrivals_by_ethcat'],
        'waiting_time_mean1':   results['waiting_time_mean1'] ,
        'waiting_time_mean2':   results['waiting_time_mean2'],
        'waiting_time_mean4':   results['waiting_time_mean4'],
        'waiting_time_mean5':   results['waiting_time_mean5'],
        'waiting_time_mean6':   results['waiting_time_mean6'],
        'waiting_time_mean7':   results['waiting_time_mean7'],
        'lr_quality_ethcat1': results['lr_quality_ethcat1'],
        'lr_quality_ethcat2': results['lr_quality_ethcat2'],
        'lr_quality_ethcat4': results['lr_quality_ethcat4'],
        'lr_quality_ethcat5': results['lr_quality_ethcat5'],
        'lr_quality_ethcat6': results['lr_quality_ethcat6'],
        'lr_quality_ethcat7': results['lr_quality_ethcat7'],
        'hr_quality_ethcat1': results['hr_quality_ethcat1'],
        'hr_quality_ethcat2': results['hr_quality_ethcat2'],
        'hr_quality_ethcat4': results['hr_quality_ethcat4'],
        'hr_quality_ethcat5': results['hr_quality_ethcat5'],
        'hr_quality_ethcat6': results['hr_quality_ethcat6'],
        'hr_quality_ethcat7': results['hr_quality_ethcat7'],
        'e_quality_ethcat1': results['e_quality_ethcat1'],
        'e_quality_ethcat2': results['e_quality_ethcat2'],
        'e_quality_ethcat4': results['e_quality_ethcat4'],
        'e_quality_ethcat5': results['e_quality_ethcat5'],
        'e_quality_ethcat6': results['e_quality_ethcat6'],
        'e_quality_ethcat7': results['e_quality_ethcat7'],
        'avg_ClassI_HLA_ethcat1': results['avg_ClassI_HLA_ethcat1'],
        'avg_ClassI_HLA_ethcat2': results['avg_ClassI_HLA_ethcat2'],
        'avg_ClassI_HLA_ethcat4': results['avg_ClassI_HLA_ethcat4'],
        'avg_ClassI_HLA_ethcat5': results['avg_ClassI_HLA_ethcat5'],
        'avg_ClassI_HLA_ethcat6': results['avg_ClassI_HLA_ethcat6'],
        'avg_ClassI_HLA_ethcat7': results['avg_ClassI_HLA_ethcat7'],
        'HLA_ClassI_total': results['HLA_ClassI_total'],

        'avg_DR_HLA_ethcat1': results['avg_DR_HLA_ethcat1'],
        'avg_DR_HLA_ethcat2': results['avg_DR_HLA_ethcat2'],
        'avg_DR_HLA_ethcat4': results['avg_DR_HLA_ethcat4'],
        'avg_DR_HLA_ethcat5': results['avg_DR_HLA_ethcat5'],
        'avg_DR_HLA_ethcat6': results['avg_DR_HLA_ethcat6'],
        'avg_DR_HLA_ethcat7': results['avg_DR_HLA_ethcat7'],
        'HLA_DR_total': results['HLA_DR_total'],
        

        'avg_DQ_HLA_ethcat1': results['avg_DQ_HLA_ethcat1'],
        'avg_DQ_HLA_ethcat2': results['avg_DQ_HLA_ethcat2'],
        'avg_DQ_HLA_ethcat4': results['avg_DQ_HLA_ethcat4'],
        'avg_DQ_HLA_ethcat5': results['avg_DQ_HLA_ethcat5'],
        'avg_DQ_HLA_ethcat6': results['avg_DQ_HLA_ethcat6'],
        'avg_DQ_HLA_ethcat7': results['avg_DQ_HLA_ethcat7'],
        'HLA_DQ_total': results['HLA_DQ_total'],

        'F_per_ethcat':   results['F_per_ethcat'],   
        'L_per_ethcat':   results['L_per_ethcat'],   
        'F_total': results['F_total'],
        'lr_quality_total': results['lr_quality_total'],
        'hr_quality_total': results['hr_quality_total'],
        'e_quality_total': results['e_quality_total'],
        'Waiting_time_mean': results['Waiting_time_mean']
    })
    

Runing simulation 1 of 100
Runing simulation 2 of 100
Runing simulation 3 of 100
Runing simulation 4 of 100
Runing simulation 5 of 100
Runing simulation 6 of 100
Runing simulation 7 of 100
Runing simulation 8 of 100
Runing simulation 9 of 100
Runing simulation 10 of 100
Runing simulation 11 of 100
Runing simulation 12 of 100
Runing simulation 13 of 100
Runing simulation 14 of 100
Runing simulation 15 of 100
Runing simulation 16 of 100
Runing simulation 17 of 100
Runing simulation 18 of 100
Runing simulation 19 of 100
Runing simulation 20 of 100
Runing simulation 21 of 100
Runing simulation 22 of 100
Runing simulation 23 of 100
Runing simulation 24 of 100
Runing simulation 25 of 100
Runing simulation 26 of 100
Runing simulation 27 of 100
Runing simulation 28 of 100
Runing simulation 29 of 100
Runing simulation 30 of 100
Runing simulation 31 of 100
Runing simulation 32 of 100
Runing simulation 33 of 100
Runing simulation 34 of 100
Runing simulation 35 of 100
Runing simulation 36 of 100
R

In [ ]:
df_results = pd.DataFrame(simulations_results)

list_HLA_ClassI_total = df_results['HLA_ClassI_total'].tolist()


list_HLA_DR_total = df_results['HLA_DR_total'].tolist()


list_HLA_DQ_total = df_results['HLA_DQ_total'].tolist()



lists_ClassI = {
    1: df_results['avg_ClassI_HLA_ethcat1'].dropna().tolist(),
    2: df_results['avg_ClassI_HLA_ethcat2'].dropna().tolist(),
    4: df_results['avg_ClassI_HLA_ethcat4'].dropna().tolist(),
    5: df_results['avg_ClassI_HLA_ethcat5'].dropna().tolist(),
    6: df_results['avg_ClassI_HLA_ethcat6'].dropna().tolist(),
    7: df_results['avg_ClassI_HLA_ethcat7'].dropna().tolist()
}

lists_DR = {
    1: df_results['avg_DR_HLA_ethcat1'].dropna().tolist(),
    2: df_results['avg_DR_HLA_ethcat2'].dropna().tolist(),
    4: df_results['avg_DR_HLA_ethcat4'].dropna().tolist(),
    5: df_results['avg_DR_HLA_ethcat5'].dropna().tolist(),
    6: df_results['avg_DR_HLA_ethcat6'].dropna().tolist(),
    7: df_results['avg_DR_HLA_ethcat7'].dropna().tolist()
}

lists_DQ = {
    1: df_results['avg_DQ_HLA_ethcat1'].dropna().tolist(),
    2: df_results['avg_DQ_HLA_ethcat2'].dropna().tolist(),
    4: df_results['avg_DQ_HLA_ethcat4'].dropna().tolist(),
    5: df_results['avg_DQ_HLA_ethcat5'].dropna().tolist(),
    6: df_results['avg_DQ_HLA_ethcat6'].dropna().tolist(),
    7: df_results['avg_DQ_HLA_ethcat7'].dropna().tolist()
}



In [306]:
import scipy.stats as st

ethnicity = [1, 2, 4, 5, 6, 7]
n_sim = len(df_results)

def media_ci(list):
    m = np.mean(list)
    s = np.std(list, ddof=1)
    low, high = st.t.interval(0.95, len(list)-1, loc=m, scale=s/np.sqrt(len(list)))
    return m, f"{m:.3f} [{low:.3f}; {high:.3f}]"

rows = []
# Reporting per ethnicity 

for e in ethnicity:
    arrivals_e = df_results['arrivals_by_ethcat'].apply(lambda d: d.get(e, 0)).sum() / n_sim
    transplants_e = df_results['cycles'].apply(
        lambda cycles: sum(1 for c in cycles for n in c if recipients.loc[n, 'ETHCAT'] == e)
    ).sum() / n_sim

    F_vals = df_results['F_per_ethcat'].apply(lambda d: d[e])
    _, txt_F = media_ci(F_vals)
    std_F = np.std(F_vals, ddof=1)

    lr_quality = df_results[f'lr_quality_ethcat{e}']
    _, txt_antigen = media_ci(lr_quality)

    hr_quality = df_results[f'hr_quality_ethcat{e}']
    _, txt_allele = media_ci(hr_quality)

    e_quality  = df_results[f'e_quality_ethcat{e}']
    _, txt_eplet = media_ci(e_quality)

    waiting = df_results[f'waiting_time_mean{e}'].mean()

    L_vals = df_results['L_per_ethcat'].apply(lambda d: d[e])
    _, txt_L = media_ci(L_vals)

    f_mean = np.mean(F_vals)
    l_mean = np.mean(L_vals)
    still_in_kep = round(1 - f_mean - l_mean, 3)


    list_classi = lists_ClassI[e]

    _, txt_classi = media_ci(list_classi)

    list_dr = lists_DR[e]
    _, txt_dr = media_ci(list_dr)

    list_dq = lists_DQ[e]
    _, txt_dq = media_ci(list_dq)


    rows.append({
        'Ethnicity(s)': e,
        'Arrivals': round(arrivals_e, 2),
        'Transplants': round(transplants_e, 2),
        'F(s) (Matched)': txt_F,
        'HLA(s) Antigen': txt_antigen,
        'HLA(s) Allele': txt_allele,
        'HLA(s) Eplets': txt_eplet,
        'Waiting Time': round(waiting, 3),
        'L(s) (Left Unmatched)': txt_L,
        '1-F(s)-L(s) (Still in KEP)': still_in_kep,
        'HLA ClassI': txt_classi,
        'HLA DR': txt_dr,
        'HLA DQ': txt_dq,

    })

# Reporting the totals

total_arrivals = sum(df_results['arrivals_by_ethcat'].apply(lambda d: sum(d.values()))) / n_sim
total_transplants = df_results['cycles'].apply(lambda cycles: sum(len(c) for c in cycles)).sum() / n_sim
F_total = total_transplants / total_arrivals


F_s_total, txt_F_total = media_ci(df_results['F_total'])

L_vals_flat = (df_results['total_departures'] / df_results['total_entries']).tolist()
L_s_total, ic_L_total = media_ci(L_vals_flat)
txt_L_total = ic_L_total

_, txt_total_lr_quality = media_ci(df_results['lr_quality_total'])
_,txt_total_hr_quality = media_ci(df_results['hr_quality_total'])
_,txt_total_e_quality= media_ci(df_results['e_quality_total'])

txt_waiting_time_mean = np.mean(df_results['Waiting_time_mean'])

F_vals_flat = [df_results['F_per_ethcat'].iloc[i][e] for i in range(n_sim) for e in ethnicity]
std_F_total = np.std(F_vals_flat, ddof=1)

f_total_mean = F_total  
total_outgoing = sum(
    df_results['L_per_ethcat'].apply(lambda d: sum(d.values()))
) / n_sim

l_total_mean = total_outgoing / total_arrivals
still_in_kep_total = round(1 - F_total - l_total_mean, 3)


_, txt_classi_total = media_ci(list_HLA_ClassI_total)

_, txt_dr_total = media_ci(list_HLA_DR_total )

_, txt_dq_total = media_ci(list_HLA_DQ_total )

rows.append({
    'Ethnicity(s)': 'Entire Population',
    'Arrivals': round(total_arrivals, 2),
    'Transplants': round(total_transplants, 2),
    'F(s) (Matched)': txt_F_total,
    'HLA(s) Antigen': txt_total_lr_quality,
    'HLA(s) Allele': txt_total_hr_quality,
    'HLA(s) Eplets': txt_total_e_quality,
    'Waiting Time': txt_waiting_time_mean,
    'L(s) (Left Unmatched)': txt_L_total,
    '1-F(s)-L(s) (Still in KEP)': f"{(1 - L_s_total - F_s_total):.3f}",
    'HLA ClassI': txt_classi_total,
    'HLA DR': txt_dr_total,
    'HLA DQ': txt_dq_total
})

df_final_table = pd.DataFrame(rows)
display(df_final_table)

,Ethnicity(s),Arrivals,Transplants,F(s) (Matched),HLA(s) Antigen,HLA(s) Allele,HLA(s) Eplets,Waiting Time,L(s) (Left Unmatched),1-F(s)-L(s) (Still in KEP),HLA ClassI,HLA DR,HLA DQ
0,1,592.24,374.92,0.633 [0.630; 0.636],2.447 [2.438; 2.456],1.663 [1.653; 1.673],53.434 [53.356; 53.512],4.183000,0.295 [0.291; 0.298],0.072,2.446 [2.436; 2.457],1.135 [1.130; 1.141],1.312 [1.307; 1.317]
1,2,169.69,110.76,0.653 [0.646; 0.660],2.261 [2.245; 2.276],1.341 [1.326; 1.356],52.418 [52.300; 52.536],4.026000,0.280 [0.273; 0.287],0.067,1.939 [1.919; 1.959],0.993 [0.984; 1.003],1.267 [1.258; 1.277]
2,4,147.11,97.18,0.661 [0.654; 0.667],2.236 [2.219; 2.253],1.334 [1.318; 1.350],52.291 [52.173; 52.408],4.199000,0.279 [0.271; 0.286],0.061,2.183 [2.162; 2.204],1.016 [1.006; 1.025],1.221 [1.209; 1.232]
3,5,55.43,34.87,0.629 [0.618; 0.640],2.138 [2.112; 2.165],1.084 [1.057; 1.111],51.416 [51.214; 51.618],4.493000,0.303 [0.292; 0.315],0.067,2.039 [2.000; 2.077],0.909 [0.892; 0.927],1.229 [1.211; 1.247]
4,6,7.93,5.39,0.680 [0.656; 0.704],2.334 [2.262; 2.405],1.453 [1.372; 1.535],52.857 [52.357; 53.357],2.972000,0.254 [0.226; 0.281],0.067,2.262 [2.160; 2.364],0.999 [0.951; 1.047],1.335 [1.279; 1.391]
5,7,5.90,2.72,0.461 [0.435; 0.487],2.030 [1.914; 2.146],1.252 [1.146; 1.359],52.904 [52.246; 53.562],4.838000,0.431 [0.401; 0.460],0.108,1.738 [1.619; 1.858],0.843 [0.782; 0.903],1.188 [1.108; 1.267]
6,Entire Population,978.30,625.84,0.640 [0.637; 0.642],2.362 [2.355; 2.368],1.519 [1.511; 1.527],52.957 [52.901; 53.013],4.171801,0.291 [0.288; 0.294],0.069,2.292 [2.284; 2.300],1.077 [1.072; 1.081],1.285 [1.281; 1.289]


In [221]:
df_final_table # este tiene la dision con P 

,Ethnicity(s),Arrivals,Transplants,F(s) (Matched),HLA(s) Antigen,HLA(s) Allele,HLA(s) Eplets,Waiting Time,L(s) (Left Unmatched),1-F(s)-L(s) (Still in KEP),HLA ClassI,HLA DR,HLA DQ
0,1,591.56,398.28,0.673 [0.669; 0.677],4.683 [4.668; 4.697],3.394 [3.379; 3.410],93.036 [92.907; 93.165],3.0950,0.290 [0.287; 0.294],0.036,2.559 [2.548; 2.570],0.940 [0.935; 0.946],1.183 [1.178; 1.189]
1,2,170.05,115.52,0.679 [0.672; 0.686],4.174 [4.148; 4.199],2.534 [2.509; 2.558],90.787 [90.567; 91.006],3.1280,0.286 [0.279; 0.293],0.035,2.138 [2.117; 2.159],0.851 [0.839; 0.863],1.184 [1.174; 1.195]
2,4,147.22,97.92,0.665 [0.658; 0.672],4.324 [4.298; 4.350],2.690 [2.663; 2.717],91.165 [90.896; 91.433],3.2640,0.299 [0.291; 0.307],0.036,2.327 [2.305; 2.349],0.875 [0.863; 0.888],1.122 [1.110; 1.133]
3,5,55.31,36.93,0.667 [0.654; 0.680],3.935 [3.900; 3.969],2.033 [2.001; 2.065],84.352 [83.947; 84.757],3.3860,0.299 [0.286; 0.312],0.033,2.116 [2.084; 2.147],0.733 [0.714; 0.752],1.086 [1.067; 1.105]
4,6,7.94,5.88,0.740 [0.714; 0.767],4.219 [4.121; 4.317],2.873 [2.765; 2.981],83.872 [82.857; 84.887],2.1020,0.234 [0.207; 0.261],0.025,2.160 [2.077; 2.242],0.856 [0.809; 0.903],1.203 [1.150; 1.256]
5,7,5.93,2.67,0.450 [0.415; 0.486],4.142 [3.998; 4.286],2.322 [2.163; 2.481],93.955 [92.581; 95.329],5.6300,0.474 [0.437; 0.512],0.075,2.081 [1.965; 2.197],0.793 [0.717; 0.869],1.268 [1.188; 1.349]
6,Entire Population,978.01,657.20,0.672 [0.669; 0.675],4.491 [4.481; 4.502],3.053 [3.041; 3.065],91.796 [91.698; 91.894],3.1462,0.292 [0.289; 0.296],0.036,2.420 [2.412; 2.429],0.902 [0.897; 0.907],1.169 [1.165; 1.173]


COMPARACION DE TABLAS

In [144]:
import numpy as np
import pandas as pd

# UTILITIES

def _canonical_cycle(cycle):
    #Return a representation of a cycle to enable set comparison.
    c = list(cycle)
    i0 = min(range(len(c)), key=lambda i: c[i])
    r = c[i0:] + c[:i0]
    rev = list(reversed(r))
    return tuple(r) if r <= rev else tuple(rev)

def _edge_weight_hr(u, v):
    #Change the resolution here
    return float(hla_lr.iloc[v, u])

def _eth_of(v):
    try:
        return int(recipients.loc[v, 'ETHCAT'])
    except Exception:
        return int(recipients.loc[recipients['Nodo'] == v, 'ETHCAT'].iloc[0])

def _edge_contrib_approx(u, v, multipliers, P_est, Z=10.0):
   
    #Approximate contribution of edge u->v to the objective:
    #  contrib ≈ m_eth * (1 + w_hr / (P_est * Z))
    eth = _eth_of(v)
    m = multipliers.get(eth, 1.0)
    w = _edge_weight_hr(u, v)
    contrib = m * (1.0 + w / (P_est * Z))
    return contrib, eth, m, w

def _cycle_contrib_table(cycle, multipliers, P_est, Z=10.0):
    rows, total = [], 0.0
    nodes = list(cycle)
    for u, v in zip(nodes, nodes[1:] + nodes[:1]):
        contrib, eth, m, w = _edge_contrib_approx(u, v, multipliers, P_est, Z)
        rows.append({'u': u, 'v': v, 'ETHCAT': eth, 'm': m, 'w_hr': w, 'contrib': contrib})
        total += contrib
    return total, pd.DataFrame(rows)

def _count_arcs_by_eth(cycles, eths=(1,2,4,5,6,7)):
    #Count edges u->v by ethnicity
    counts = {e: 0 for e in eths}
    for cyc in cycles:
        nodes = list(cyc)
        for u, v in zip(nodes, nodes[1:] + nodes[:1]):
            eth = _eth_of(v)
            if eth in counts:
                counts[eth] += 1
    return counts

def _summarize_one_result(res_dict):
    cycles = res_dict['historial_cycles']
    selected_cycles = len(cycles)
    selected_arcs   = sum(len(c) for c in cycles)
    arcs_by_eth = _count_arcs_by_eth(cycles)

    out = {
        'selected_cycles': selected_cycles,
        'selected_arcs': selected_arcs,
        'F_total': res_dict.get('F_total', np.nan),
        'Waiting_time_mean': res_dict.get('Waiting_time_mean', np.nan),
        'lr_quality_total': res_dict.get('lr_quality_total', np.nan),
        'hr_quality_total': res_dict.get('hr_quality_total', np.nan),
        'e_quality_total':  res_dict.get('e_quality_total',  np.nan),
    }
    for e, c in arcs_by_eth.items():
        out[f'arcs_eth{e}'] = c
    return out

# RUN MANY SIMULATIONS WITH CONTROLLED SEEDS


def run_many(multipliers, n_sims=100, base_seed=12345):
    
    results, rows = [], []
    for i in range(n_sims):
        np.random.seed(base_seed + i)  # same randomness per index
        res = run_simulation(
            total_time=10*12,
            arrival_rate=(990/(10*12)),
            departure_rate=((990/(10*12))*0.29),
            match_run=3,
            pairs=pairs,
            compatibility=compatibility,
            hla_lr=hla_lr,
            hla_hr=hla_hr,
            hla_eplet=hla_eplet,
            multipliers_ethnicity=multipliers
        )
        results.append(res)
        row = {'sim': i}
        row.update(_summarize_one_result(res))
        rows.append(row)
    summary_df = pd.DataFrame(rows)
    return results, summary_df

# BASE vs MODIFIED COMPARISON

def compare_scenarios_via_sim(
    multipliers_base=None,
    multipliers_mod=None,
    n_sims=100,
    seed=12345,
    Z=10.0
):

    if multipliers_base is None:
        multipliers_base = {1:1.0, 2:1.0, 4:1.0, 5:1.0, 6:1.0, 7:1.0}
    if multipliers_mod is None:
        multipliers_mod  = {1:0.998, 2:1.0, 4:1.0, 5:1.002, 6:1.0, 7:1.0}

    base_results, base_df = run_many(multipliers_base, n_sims=n_sims, base_seed=seed)
    mod_results,  mod_df  = run_many(multipliers_mod,  n_sims=n_sims, base_seed=seed)

    # 1) means across simulations
    metrics = ['selected_cycles', 'selected_arcs', 'F_total', 'Waiting_time_mean',
               'lr_quality_total', 'hr_quality_total', 'e_quality_total']
    comp_rows = []
    for m in metrics:
        b_mean = float(base_df[m].mean())
        d_mean = float(mod_df[m].mean())
        comp_rows.append({'metric': m, 'base': b_mean, 'modified': d_mean, 'delta': d_mean - b_mean})
    metrics_comp = pd.DataFrame(comp_rows)

    # selected arcs by ethnicity 
    eth_list = [1,2,4,5,6,7]
    arcs_rows = []
    for e in eth_list:
        col = f'arcs_eth{e}'
        b_mean = float(base_df[col].mean()) if col in base_df.columns else float('nan')
        d_mean = float(mod_df[col].mean())  if col in mod_df.columns  else float('nan')
        arcs_rows.append({'ethcat': e, 'base': b_mean, 'modified': d_mean, 'delta': d_mean - b_mean})
    arcs_by_eth_comp = pd.DataFrame(arcs_rows)

    common_info = None
    only_base, only_mod = None, None
    common_cycle_table = None
    P_est = len(pairs.index)

    for i in range(n_sims):
        cyc_b = base_results[i]['historial_cycles']
        cyc_m = mod_results[i]['historial_cycles']
        set_b = { _canonical_cycle(c) for c in cyc_b }
        set_m = { _canonical_cycle(c) for c in cyc_m }
        inter = list(set_b & set_m)
        if inter:
            cyc = list(inter[0])
            total_b, tbl_b = _cycle_contrib_table(cyc, multipliers_base, P_est, Z=Z)
            total_m, tbl_m = _cycle_contrib_table(cyc, multipliers_mod,  P_est, Z=Z)
            common_cycle_table = tbl_b.copy()
            common_cycle_table.rename(columns={'m':'m_base','contrib':'contrib_base'}, inplace=True)
            common_cycle_table['m_mod']         = tbl_m['m'].values
            common_cycle_table['contrib_mod']   = tbl_m['contrib'].values
            common_cycle_table['delta_contrib'] = common_cycle_table['contrib_mod'] - common_cycle_table['contrib_base']

            common_info = {
                'sim_index': i,
                'cycle': tuple(cyc),
                'total_base': float(total_b),
                'total_mod': float(total_m),
                'delta_total': float(total_m - total_b),
            }
            only_base = list(set_b - set_m)
            only_mod  = list(set_m - set_b)
            break

    return metrics_comp, arcs_by_eth_comp, common_info, common_cycle_table, only_base, only_mod

# EXAMPLE RUN

m_base = {1:1.0,   2:1.0, 4:1.0, 5:1.0,   6:1.0, 7:1.0}
m_mod  = {1:0.998, 2:1.0, 4:1.001, 5:1.001, 6:1.0, 7:1.0}

metrics_df, arcs_by_eth_df, common_info, common_cycle_df, only_base, only_mod = compare_scenarios_via_sim(
    multipliers_base=m_base,
    multipliers_mod=m_mod,
    n_sims=100,
    seed=12345,
    Z=10.0
)

print("\n=== Comparative metrics (mean across simulations) ===")
print(metrics_df.to_string(index=False, float_format=lambda x: f"{x:.6f}" if isinstance(x,float) else str(x)))

print("\n=== Selected arcs by ethnicity (mean per simulation) ===")
print(arcs_by_eth_df.to_string(index=False, float_format=lambda x: f"{x:.6f}" if isinstance(x,float) else str(x)))

if common_info is not None:
    print(f"\n=== Common cycle (sim={common_info['sim_index']}) ===")
    print(f"Cycle: {common_info['cycle']}")
    print(f"Approx. total (base):    {common_info['total_base']:.6f}")
    print(f"Approx. total (modified):{common_info['total_mod']:.6f}")
    print(f"Delta total:             {common_info['delta_total']:+.6f}")
    print("\nPer-edge breakdown (u->v):")
    print(common_cycle_df.to_string(index=False, float_format=lambda x: f'{x:.6f}' if isinstance(x,float) else str(x)))
else:
    print("\n(No common cycle found in the inspected simulations.)")

if only_base:
    print(f"\nCycles ONLY in base (examples): {only_base[:5]}")
if only_mod:
    print(f"Cycles ONLY in modified (examples): {only_mod[:5]}")



=== Comparative metrics (mean across simulations) ===
           metric       base   modified     delta
  selected_cycles 257.920000 257.150000 -0.770000
    selected_arcs 656.420000 655.740000 -0.680000
          F_total   0.671341   0.670171 -0.001171
Waiting_time_mean   3.156223   3.149267 -0.006956
 lr_quality_total   4.516773   4.486849 -0.029923
 hr_quality_total   3.085509   3.053586 -0.031922
  e_quality_total  91.923726  91.734340 -0.189386

=== Selected arcs by ethnicity (mean per simulation) ===
 ethcat       base   modified      delta
      1 414.130000 399.290000 -14.840000
      2 107.960000 112.270000   4.310000
      4  91.650000  98.230000   6.580000
      5  34.040000  36.940000   2.900000
      6   6.040000   6.110000   0.070000
      7   2.600000   2.900000   0.300000

=== Common cycle (sim=0) ===
Cycle: (735, 756)
Approx. total (base):    2.000808
Approx. total (modified):1.996806
Delta total:             -0.004002

Per-edge breakdown (u->v):
  u   v  ETHCAT   m_b

##Comparación entre escenario original y escenario modificado

In [ ]:
### FOR LOW RESOLUTION ESCENARIO
#m_base = {1:1.0,   2:1.0, 4:1.0, 5:1.0,   6:1.0, 7:1.0}
#m_mod  = {1:0.998, 2:1.0, 4:1.001, 5:1.001, 6:1.0, 7:1.0}




In [ ]:
### FOR HIGH RESOLUTION ESCENARIO
#m_base = {1:1.0,   2:1.0, 4:1.0, 5:1.0,   6:1.0, 7:1.0}
#m_mod  = {1:0.998, 2:1.0, 4:1.0, 5:1.002, 6:1.0, 7:1.0}

"""

=== Selected arcs by ethnicity (mean per simulation) ===
 ethcat       base   modified      delta
      1 411.380000 400.540000 -10.840000
      2 105.420000 112.020000   6.600000
      4  88.730000  94.500000   5.770000
      5  30.140000  34.430000   4.290000
      6   6.450000   6.670000   0.220000
      7   2.060000   2.110000   0.050000
"""


In [ ]:
### FOR EPLET ESCENARIO
